In [1]:
from bs4 import BeautifulSoup
import requests

resp = requests.get('https://www.ura.gov.sg/realEstateIIWeb/transaction/search.action')

soup = BeautifulSoup(resp.text, 'lxml')

In [2]:
for item in soup.find(id='select1').children:
    item_string = item.string
    if item_string != '\n':
        print(item_string)

01
02
03
04
05
06
07
08
09
10
11
12
13
14
15
16
17
18
19
20
21
22
23
25
26
27
28


In [3]:
dt_list = []
for item in soup.find(id='searchForm_selectedFromPeriodPostalDistrict').contents:
    item_string = item.string
    if item_string != '\n':
        dt_list.append(item_string)

from datetime import datetime
datetime.strftime(min([datetime.strptime(dt, '%b %Y') for dt in dt_list]), '%b %Y')
    

'Oct 2015'

In [47]:
dt_list = []
for item in soup.find(id='searchForm_selectedToPeriodPostalDistrict').contents:
    item_string = item.string
    if item_string != '\n':
        dt_list.append(item_string)

from datetime import datetime
datetime.strftime(max([datetime.strptime(dt, '%b %Y') for dt in dt_list]), '%b %Y')
    

'Oct 2020'

In [11]:

get_url = 'https://www.ura.gov.sg/realEstateIIWeb/transaction/search.action'
post_url = 'https://www.ura.gov.sg/realEstateIIWeb/transaction/submitSearch.action'
session = requests.Session()
session.get(get_url)


<Response [200]>

In [28]:
import html5lib
cookies = requests.utils.dict_from_cookiejar(session.cookies)
cookie_post_url = post_url + ';jsessionid=' + cookies['JSESSIONID']

print(cookie_post_url)
data = {
    'submissionType': 'pd',
    'selectedFromPeriodProjectName': 'OCT 2015',
    'selectedToPeriodProjectName': "OCT 2020",
    '__multiselect_selectedProjects1': "",
    'selectedFromPeriodPostalDistrict': "OCT 2015",
    'selectedToPeriodPostalDistrict': "OCT 2020",
    'propertyType': "ac",
    'postalDistrictList': "28",
    'selectedPostalDistricts1': ['01'],
    '__multiselect_selectedPostalDistricts1': ''
}
resp = session.post(cookie_post_url, data)


https://www.ura.gov.sg/realEstateIIWeb/transaction/submitSearch.action;jsessionid=w9OB0DNxl1RIwGT-WhYVfzWoLvJiwO8hSkM-6MDcwfHcD0i6NKw7!655280793!585094586


In [39]:
import pandas as pd
df = pd.read_html(resp.text)[0]
df

,Project Name,Street Name,Type,Postal District,Market Segment,Tenure,Type of Sale,No. of Units,Price ($),Nett Price ($),Area (Sqft)¹,Type of Area²,Floor Level,Unit Price ($psf)³,Date of Sale⁴
0,THE SAIL @ MARINA BAY,MARINA BOULEVARD,Apartment,1,CCR,99 yrs lease commencing from 2002,Resale,1,970000,-,592,Strata,06 to 10,1638,Oct-20
1,MARINA ONE RESIDENCES,MARINA WAY,Apartment,1,CCR,99 yrs lease commencing from 2011,Resale,1,1772714,-,753,Strata,26 to 30,2353,Oct-20
2,MARINA ONE RESIDENCES,MARINA WAY,Apartment,1,CCR,99 yrs lease commencing from 2011,Resale,1,2921233,-,1141,Strata,26 to 30,2560,Oct-20
3,MARINA ONE RESIDENCES,MARINA WAY,Apartment,1,CCR,99 yrs lease commencing from 2011,Resale,1,1765997,-,753,Strata,21 to 25,2344,Oct-20
4,THE SAIL @ MARINA BAY,MARINA BOULEVARD,Apartment,1,CCR,99 yrs lease commencing from 2002,Resale,1,1620000,-,861,Strata,11 to 15,1881,Oct-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1032,PEOPLE'S PARK COMPLEX,PARK ROAD,Apartment,1,RCR,99 yrs lease commencing from 1968,Resale,1,518000,-,409,Strata,11 to 15,1266,Nov-15
1033,THE SAIL @ MARINA BAY,MARINA BOULEVARD,Apartment,1,CCR,99 yrs lease commencing from 2002,Resale,1,1560000,-,883,Strata,41 to 45,1767,Nov-15
1034,THE SAIL @ MARINA BAY,MARINA BOULEVARD,Apartment,1,CCR,99 yrs lease commencing from 2002,Resale,1,3600000,-,1755,Strata,61 to 65,2052,Nov-15
1035,PEOPLE'S PARK COMPLEX,PARK ROAD,Apartment,1,RCR,99 yrs lease commencing from 1968,Resale,1,905000,-,1119,Strata,26 to 30,808,Nov-15


In [40]:
df.columns

Index(['Project Name', 'Street Name', 'Type', 'Postal District',
       'Market Segment', 'Tenure', 'Type of Sale', 'No. of  Units',
       'Price  ($)', 'Nett Price  ($)', 'Area (Sqft)¹', 'Type of Area²',
       'Floor Level', 'Unit Price ($psf)³', 'Date of Sale⁴'],
      dtype='object')

In [43]:
columns_mapping = {
    'Project Name': 'project_name',
    'Street Name': 'street_name',
    'Type': 'type',
    'Postal District': 'postal_district',
    'Market Segment': 'market_segment',
    'Tenure': 'tenure',
    'Type of Sale': 'type_of_sale',
    'No. of  Units': 'num_units',
    'Price  ($)': 'price',
    'Nett Price  ($)': 'nett_price',
    'Area (Sqft)¹': 'area_sqft',
    'Type of Area²': 'type_of_area',
    'Floor Level': 'floor',
    'Unit Price ($psf)³': 'unit_price_psf',
    'Date of Sale⁴': 'reference_period',
}
df.rename(columns=columns_mapping, inplace=True)
df['reference_period'] = pd.to_datetime(df['reference_period'], format='%b-%y')
df['reference_period'] = df['reference_period'] + pd.DateOffset(months=1, days=-1) 
df

,project_name,street_name,type,postal_district,market_segment,tenure,type_of_sale,num_units,price,nett_price,area_sqft,type_of_area,floor,unit_price_psf,reference_period
0,THE SAIL @ MARINA BAY,MARINA BOULEVARD,Apartment,1,CCR,99 yrs lease commencing from 2002,Resale,1,970000,-,592,Strata,06 to 10,1638,2020-11-29
1,MARINA ONE RESIDENCES,MARINA WAY,Apartment,1,CCR,99 yrs lease commencing from 2011,Resale,1,1772714,-,753,Strata,26 to 30,2353,2020-11-29
2,MARINA ONE RESIDENCES,MARINA WAY,Apartment,1,CCR,99 yrs lease commencing from 2011,Resale,1,2921233,-,1141,Strata,26 to 30,2560,2020-11-29
3,MARINA ONE RESIDENCES,MARINA WAY,Apartment,1,CCR,99 yrs lease commencing from 2011,Resale,1,1765997,-,753,Strata,21 to 25,2344,2020-11-29
4,THE SAIL @ MARINA BAY,MARINA BOULEVARD,Apartment,1,CCR,99 yrs lease commencing from 2002,Resale,1,1620000,-,861,Strata,11 to 15,1881,2020-11-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1032,PEOPLE'S PARK COMPLEX,PARK ROAD,Apartment,1,RCR,99 yrs lease commencing from 1968,Resale,1,518000,-,409,Strata,11 to 15,1266,2015-12-29
1033,THE SAIL @ MARINA BAY,MARINA BOULEVARD,Apartment,1,CCR,99 yrs lease commencing from 2002,Resale,1,1560000,-,883,Strata,41 to 45,1767,2015-12-29
1034,THE SAIL @ MARINA BAY,MARINA BOULEVARD,Apartment,1,CCR,99 yrs lease commencing from 2002,Resale,1,3600000,-,1755,Strata,61 to 65,2052,2015-12-29
1035,PEOPLE'S PARK COMPLEX,PARK ROAD,Apartment,1,RCR,99 yrs lease commencing from 1968,Resale,1,905000,-,1119,Strata,26 to 30,808,2015-12-29


In [45]:
columns_mapping.values()

dict_values(['project_name', 'street_name', 'type', 'postal_district', 'market_segment', 'tenure', 'type_of_sale', 'num_units', 'price', 'nett_price', 'area_sqft', 'type_of_area', 'floor', 'unit_price_psf', 'reference_period'])